In [11]:
pip install transformers langchain pinecone-client openai datasets

Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/ed/a5/33cf000137545a08b0a3a6ea76c8ccbd87917f78bb5d737f9f56f3b11ef6/datasets-3.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for tqdm>=4.27 from https://files.pythonhosted.org/packages/d0/30/dc54f88dd4a2b5dc8a0279bdd7270e735851848b762aeb1c1184ed1f6b14/tqdm-4.67.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 3.0 MB/s eta 0:00:00
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/52/1c/fa3b61c0cf03e1da4767213672efe186b1dfa4fc901a4a694fb184a513d1/xxhash-3.5.0-cp311-cp

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


In [12]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, GPT2Config
from datasets import load_dataset

# Load pre-trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load your domain-specific dataset (e.g., medical dataset)
dataset = load_dataset('Symptom2Disease.csv')  # Replace with actual path or dataset name

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    save_steps=500,
    eval_steps=500,
    warmup_steps=200,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

C:\Users\vipul patil\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vipul patil\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

DatasetNotFoundError: Dataset 'Symptom2Disease.csv' doesn't exist on the Hub or cannot be accessed.

In [15]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key="pcsk_5GYuPi_LLpNfJewECArYBRRdFyV3DEWqsek4fSazUSVfAL8hNYRknuR5g5qzs7F2DoDRT7")

# Check if the index already exists
if 'medical-database' not in pc.list_indexes().names():
    # Create a new index if it does not exist
    pc.create_index(
        name='medical-database',
        dimension=1536,  # Adjust according to your embedding model's dimension
        metric='euclidean',
        spec=ServerlessSpec(cloud='aws', region='us-west-2')  # Specify your region
    )

# Now that the index is ready, we can connect to it
index = pc.index('medical-database')


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'dc85a1ea4bb1c672e43d6addc9c89ff8', 'Date': 'Sat, 07 Dec 2024 23:23:40 GMT', 'Server': 'Google Frontend', 'Content-Length': '200', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Bad request: Your free plan does not support indexes in the us-west-2 region of aws. To create indexes in this region, upgrade your plan."},"status":400}


In [14]:
import pinecone
from langchain.embeddings import OpenAIEmbeddings

# Initialize Pinecone (make sure you have Pinecone API key set up)
pinecone.init(api_key="pcsk_5GYuPi_LLpNfJewECArYBRRdFyV3DEWqsek4fSazUSVfAL8hNYRknuR5g5qzs7F2DoDRT7")
index = pinecone.Index("medical-database")

# Initialize OpenAI Embeddings for symptom-text embeddings
embedding_model = OpenAIEmbeddings()

# Example function to add data to Pinecone
def add_data_to_pinecone(text_data):
    # Generate embeddings for the text
    embeddings = embedding_model.embed_documents(text_data)
    
    # Upsert (add or update) data in Pinecone
    index.upsert(vectors=embeddings)

# Example medical data (you should replace with real medical data)
medical_data = [
    "Pneumonia is an infection of the lungs.",
    "Common symptoms of pneumonia include cough, fever, and difficulty breathing."
]

# Add data to Pinecone
add_data_to_pinecone(medical_data)


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )

